In [1]:
import os 
import torch
print(torch.__version__)
# os.environ["CUDA_VISIBLE_DEVICES"] = '1'
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'  # For synchronous execution of CPU and GPU
client_device = torch.device("cuda:2")
server_device = torch.device("cuda:1")
model_device = torch.device("cuda:0")
from transformers import OPTForQuestionAnswering

1.13.1+cu116


/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from transformers import GPT2LMHeadModel
import torch
from typing import Optional, List
from torch.nn import CrossEntropyLoss
from transformers.modeling_outputs import CausalLMOutputWithPast


class ClientSideGPT2LMHeadModel(GPT2LMHeadModel):
    def __init__(self, config):
        super().__init__(config)


class ServerSideGPT2LMHeadModel(GPT2LMHeadModel):
    def __init__(self, config):
        super().__init__(config)
        split_layer = 2
 
        

In [ ]:
from transformers import OPTConfig
config = OPTConfig(num_hidden_layers=1)
client_model = ClientSideOPTForCausalLM(config)
num_hidden_layers=11
config = OPTConfig()
server_model = ServerSideOPTForCausalLM(config)
    

In [ ]:
print(client_model)

In [ ]:
print(server_model)

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model
import torch

MODEL_NAME = "facebook/opt-125m"
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME, 
    trust_remote_code=True,
    cache_dir="/app/.huggingface_cache/model/"
)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    cache_dir="/app/.huggingface_cache/model/",
    trust_remote_code=True,
    torch_dtype=torch.float16
)


lora_modules=["q_proj", "k_proj", "v_proj", "out_proj", "fc1", "fc2", "lm_head"] 
lora_config = LoraConfig(
    r=1,  # dimension of the updated matrices
    lora_alpha=64,  # parameter for scaling
    target_modules=lora_modules,
    lora_dropout=0.1,  # dropout probability for layers
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, lora_config)
client_model = get_peft_model(client_model, lora_config)
server_model = get_peft_model(server_model, lora_config)

In [ ]:
print(model)

In [ ]:
print(server_model)

In [ ]:
print(client_model)

In [ ]:
from datasets import load_dataset
from transformers import default_data_collator
from data_utils import transform_data_to_fedml_format, group_texts, tokenize_function
from functools import partial
from torch.utils.data import DataLoader
import torch

tokenizer = AutoTokenizer.from_pretrained(
    "facebook/opt-125m", 
    trust_remote_code=True,
    cache_dir="/app/.huggingface_cache/model/"
)
inputs = tokenizer("Hello world!",return_tensors="pt")
print(inputs)

# resize embeddings
embedding_size = client_model.get_input_embeddings().weight.shape[0]
if len(tokenizer) > embedding_size:
    client_model.resize_token_embeddings(len(tokenizer))


# load data
print(tokenizer)
block_size = 1024
raw_datasets = load_dataset(
    "wikitext",
    "wikitext-2-raw-v1",
    cache_dir="/app/.huggingface_cache/dataset/",
    streaming=False
)
print(raw_datasets)
column_names = list(raw_datasets["train"].features)
print(column_names)

# data preprocessing 
__tokenize_function = partial(tokenize_function, text_column_name='text', tokenizer=tokenizer)
tokenized_datasets = raw_datasets.map(
                    __tokenize_function,
                    batched=True,
                    remove_columns=column_names,
                    desc="Running tokenizer on dataset",
                )
print(tokenized_datasets)

__group_texts = partial(group_texts, block_size=block_size)
lm_datasets = tokenized_datasets.map(
                __group_texts,
                batched=True,
                # num_proc=1,
                # load_from_cache_file=not data_args.overwrite_cache,
                # desc=f"Grouping texts in chunks of {block_size}",
            )

print(lm_datasets['train'])
lm_datasets['train'].set_format("torch", device=client_device)
train_dataloader = DataLoader(lm_datasets['train'], shuffle=True, collate_fn=default_data_collator, batch_size=1)
print(train_dataloader)
no_decay = ["bias", "layer_norm.weight"]
client_grouped_parameters = [
    {
        "params": [p for n, p in client_model.named_parameters() if not any(nd in n for nd in no_decay)],
        "weight_decay": 0.01,
    },
    {
        "params": [p for n, p in client_model.named_parameters() if any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
]
server_grouped_parameters = [
    {
        "params": [p for n, p in server_model.named_parameters() if not any(nd in n for nd in no_decay)],
        "weight_decay": 0.01,
    },
    {
        "params": [p for n, p in server_model.named_parameters() if any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
]
model_grouped_paramters = [
    {
        "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
        "weight_decay": 0.01,
    },
    {
        "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
]
server_optimizer = torch.optim.AdamW(server_grouped_parameters, lr=0.001)
client_optimizer = torch.optim.AdamW(client_grouped_parameters, lr=0.001)
model_optimizer = torch.optim.AdamW(model_grouped_paramters, lr=0.001)

In [ ]:
def unfreeze_params(model):
    for param in model.parameters():
        param.requires_grad = True
        
def print_trainable_params(model):
    for name, param in model.named_parameters():
        if(param.requires_grad):
            print(name, param.requires_grad)
        
def print_grad(model):
    for name, param in model.named_parameters():
        if(param.requires_grad):
            print(name, param.grad)
            
# unfreeze_params(client_model)
# unfreeze_params(server_model)
print("\n\n---- trainable params of server_model ----")
print_trainable_params(server_model)
print("\n\n---- trainable params of client_model ----")
print_trainable_params(client_model)
print(f"\n {len(train_dataloader)}")

In [ ]:
import math
import time
import numpy as np

print_grad = False
model.train()
client_model.train()
server_model.train()
print(len(train_dataloader))

latency = {
    "client": [],
    "server": [],
    "end-to-end": [],
    "model": []
}


scaler = torch.cuda.amp.GradScaler()
for step, batch in enumerate(train_dataloader):
    print(f"step = {step} ")
    
    for key in batch.keys():
        batch[key] = batch[key].to(device=client_device)
    
    client_model, server_model = client_model.to(dtype=torch.float16, device=client_device), server_model.to(dtype=torch.float16, device=server_device)
    # Split training
    # with torch.autocast(server_model.device.type, dtype=torch.float16, enabled=True):
    client_start_time = time.perf_counter()
    # input_ids, past_key_values, attention_mask, labels, acts = client_model(**batch)
    split_output, attention_mask, head_mask, past_key_values, inputs_embeds, labels, use_cache, output_attentions, output_hidden_states, return_dict = client_model(**batch)
    client_end_time = time.perf_counter()
    split_output, attention_mask, head_mask, past_key_values, inputs_embeds, labels, use_cache, output_attentions, output_hidden_states, return_dict
    
    split_output = split_output.to(server_device)
    # acts.retain_grad()
    # input_ids, attention_mask, labels, acts = input_ids.to(device=server_device), attention_mask.to(device=server_device), labels.to(device=server_device), acts.to(device=server_device)
    # split_output, attention_mask, head_mask, past_key_values, inputs_embeds, labels, use_cache, output_attentions, output_hidden_states, return_dict = split_output, attention_mask, head_mask, past_key_values, inputs_embeds, labels, use_cache, output_attentions, output_hidden_states, return_dict.to(server_device)
    
    server_start_time = time.perf_counter()
    # split_outputs = server_model(input_ids=input_ids, past_key_values=past_key_values, attention_mask=attention_mask, labels=labels, hidden_layer_input=acts)
    split_outputs = server_model()
    server_end_time = time.perf_counter()
        
    split_loss = split_outputs.loss
    # scaler.scale(split_loss).backward()
    split_loss.backward()
    split_perplexity = math.exp(split_loss)
    
    # scaler.step(server_optimizer)
    # scaler.step(client_optimizer)
    server_optimizer.step()
    client_optimizer.step()
    
    # scaler.update()
    if(print_grad):
        print("\n\n---- grad on server_model ----")
        print_grad(server_model)    
        print("\n\n---- grad on client_model ----")
        print_grad(client_model)
    
    latency["client"].append(client_end_time-client_start_time)
    latency["server"].append(server_end_time-server_start_time)
    latency["end-to-end"].append(server_end_time-client_start_time)
    print(f"  - split_loss = {split_loss}")
    print(f"  - split_perplexity = {split_perplexity}")
    print(f"  - Latency (sec): Client = {np.average(latency['client'])}  ||  Server = {np.average(latency['server'])}  || End-to-end = {np.average(latency['end-to-end'])}")
    
    server_optimizer.zero_grad()
    client_optimizer.zero_grad()